# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0edae08d30>
├── 'a' --> tensor([[ 0.3096, -0.9207, -1.7093],
│                   [ 1.9448,  0.3903,  0.2436]])
└── 'x' --> <FastTreeValue 0x7f0edae08a30>
    └── 'c' --> tensor([[-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                        [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                        [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00]])

In [4]:
t.a

tensor([[ 0.3096, -0.9207, -1.7093],
        [ 1.9448,  0.3903,  0.2436]])

In [5]:
%timeit t.a

63.8 ns ± 0.0508 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0edae08d30>
├── 'a' --> tensor([[-1.1852,  0.2157, -0.8025],
│                   [-0.0460, -1.1407,  0.0466]])
└── 'x' --> <FastTreeValue 0x7f0edae08a30>
    └── 'c' --> tensor([[-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                        [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                        [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00]])

In [7]:
%timeit t.a = new_value

60.6 ns ± 0.0101 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3096, -0.9207, -1.7093],
               [ 1.9448,  0.3903,  0.2436]]),
    x: Batch(
           c: tensor([[-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                      [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                      [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3096, -0.9207, -1.7093],
        [ 1.9448,  0.3903,  0.2436]])

In [11]:
%timeit b.a

57 ns ± 0.0677 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0643, -1.0075, -0.3856],
               [ 0.6273, -0.0457, -1.2835]]),
    x: Batch(
           c: tensor([[-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                      [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                      [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00]]),
       ),
)

In [13]:
%timeit b.a = new_value

476 ns ± 0.137 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

841 ns ± 0.185 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 29.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 5.83 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 274 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0eb4284b50>
├── 'a' --> tensor([[[ 0.3096, -0.9207, -1.7093],
│                    [ 1.9448,  0.3903,  0.2436]],
│           
│                   [[ 0.3096, -0.9207, -1.7093],
│                    [ 1.9448,  0.3903,  0.2436]],
│           
│                   [[ 0.3096, -0.9207, -1.7093],
│                    [ 1.9448,  0.3903,  0.2436]],
│           
│                   [[ 0.3096, -0.9207, -1.7093],
│                    [ 1.9448,  0.3903,  0.2436]],
│           
│                   [[ 0.3096, -0.9207, -1.7093],
│                    [ 1.9448,  0.3903,  0.2436]],
│           
│                   [[ 0.3096, -0.9207, -1.7093],
│                    [ 1.9448,  0.3903,  0.2436]],
│           
│                   [[ 0.3096, -0.9207, -1.7093],
│                    [ 1.9448,  0.3903,  0.2436]],
│           
│                   [[ 0.3096, -0.9207, -1.7093],
│                    [ 1.9448,  0.3903,  0.2436]]])
└── 'x' --> <FastTreeValue 0x7f0eb4284c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 57.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0ed8d29a60>
├── 'a' --> tensor([[ 0.3096, -0.9207, -1.7093],
│                   [ 1.9448,  0.3903,  0.2436],
│                   [ 0.3096, -0.9207, -1.7093],
│                   [ 1.9448,  0.3903,  0.2436],
│                   [ 0.3096, -0.9207, -1.7093],
│                   [ 1.9448,  0.3903,  0.2436],
│                   [ 0.3096, -0.9207, -1.7093],
│                   [ 1.9448,  0.3903,  0.2436],
│                   [ 0.3096, -0.9207, -1.7093],
│                   [ 1.9448,  0.3903,  0.2436],
│                   [ 0.3096, -0.9207, -1.7093],
│                   [ 1.9448,  0.3903,  0.2436],
│                   [ 0.3096, -0.9207, -1.7093],
│                   [ 1.9448,  0.3903,  0.2436],
│                   [ 0.3096, -0.9207, -1.7093],
│                   [ 1.9448,  0.3903,  0.2436]])
└── 'x' --> <FastTreeValue 0x7f0eb428bbe0>
    └── 'c' --> tensor([[-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                        [-8.1156e-01,  3.5386e-02,  5.1775e-01,

In [23]:
%timeit t_cat(trees)

29.9 µs ± 51.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.3 µs ± 177 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                       [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                       [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00]],
              
                      [[-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                       [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                       [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00]],
              
                      [[-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                       [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                       [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00]],
              
                      [[-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                       [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                       [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00]],
      

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                      [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                      [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00],
                      [-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                      [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                      [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00],
                      [-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                      [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                      [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00],
                      [-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.8608e+00],
                      [-8.1156e-01,  3.5386e-02,  5.1775e-01, -1.0318e+00],
                      [ 4.0308e-01, -5.9097e-02,  1.0404e+00,  1.6814e+00],
                      [-1.7604e-03, -7.1210e-01,  1.2323e+00,  1.86

In [28]:
%timeit Batch.cat(batches)

143 µs ± 616 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

314 µs ± 4.25 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
